## UnSu Unet Object Localization

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.initializers import he_uniform

import numpy as np

2024-10-17 21:11:08.448061: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-17 21:11:08.677062: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-17 21:11:08.678744: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-17 21:11:09.944152: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Import os and sys to allow us to customize our path to import our custom functions
import os
import sys

# # Add the top directory to the path so we can import the data module
# sys.path.append(os.path.join(os.path.join('..')))

# Import custom functions to extract our Image arrays and Pixel Mask arrays from our created fits files dataset
from dataGathering import extractImageArray, extractPixelMaskArray, extract_star_catalog

# Import astropy to read fits files
from astropy.io import fits

In [3]:
# Create images and masks arrays lists
images = []
masks = []

# Create df to store the star data inside each fits file
star_data = []

# Create a list of all the fits files in the dataset folder
fits_files = os.listdir('data/')

# For all the fits files in the dataset folder specified in file_path, extract the image and mask arrays to the respective lists
file_path = 'data/'
for file in os.listdir(file_path):
    if file.endswith('.fits'):
        images.append(extractImageArray(file_path + file))
        masks.append(extractPixelMaskArray(file_path + file))
        star_data.append(extract_star_catalog(file_path + file))

        print(file + ' added to dataset')

star16.fits added to dataset
star4.fits added to dataset
star3.fits added to dataset
star1.fits added to dataset
star13.fits added to dataset
star12.fits added to dataset
star0.fits added to dataset
star14.fits added to dataset
star8.fits added to dataset
star2.fits added to dataset
star11.fits added to dataset
star5.fits added to dataset
star18.fits added to dataset
star7.fits added to dataset
star17.fits added to dataset
star19.fits added to dataset
star6.fits added to dataset
star15.fits added to dataset
data0.fits added to dataset
star9.fits added to dataset
star10.fits added to dataset


In [6]:
images[5].shape

(300, 300)

In [4]:
input_shape = (300,300,3)

In [5]:
# Encoder path
    #32 kernels
    #3x3 kernel size
    #padding = same considers edges in the input
def unet_model(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    # Encoder path
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(inputs)
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(p1)
    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(p2)
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(p3)
    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(p4)
    c5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(c5)

    # Decoder path
    u6 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(u6)
    c6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(c6)

    u7 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(u7)
    c7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(c7)

    u8 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(u8)
    c8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(c8)

    u9 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(u9)
    c9 = layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(c9)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9) # 300x300x1 filled with 1s or 0s

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model


In [6]:

model = unet_model(input_shape)
model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy']
)

2024-10-17 20:59:35.844917: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-17 20:59:35.845421: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 36, 36, 256), (None, 37, 37, 256)]

In [ ]:
# train_images and train_masks are training data and masks

In [ ]:
history = model.fit(train_images, train_masks, 
                    validation_data=(val_images, val_masks), 
                    epochs=100, 
                    batch_size=32)

In [ ]:



# Set a larger figure size
plt.figure(figsize=(10, 6))

# Plot the training and validation loss
plt.plot(history.history['loss'], color='blue', linestyle='-', linewidth=2, label='Training Loss')
plt.plot(history.history['val_loss'], color='orange', linestyle='--', linewidth=2, label='Validation Loss')

# Add title and labels
plt.title('Training and Validation Loss Over Epochs', fontsize=16)
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Loss', fontsize=14)

# Add a grid for better readability
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Add legend to differentiate between training and validation loss
plt.legend(loc='best', fontsize=12)

# Set limits for better visualization
plt.xlim(0, len(history.history['loss']) - 1)  # From epoch 0 to the last epoch
plt.ylim(min(history.history['loss']) * 0.95, max(history.history['val_loss']) * 1.05)

# Show the plot
plt.tight_layout()
plt.show()
